**Import Libraries**

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Load Data**

In [4]:
# Load the dataset
data = pd.read_csv('/spam.csv', encoding='latin-1')
# Check the first few rows
data.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


**Data Preprocessing**

In [8]:

data = pd.read_csv('/spam.csv', encoding='latin-1')

# Check the first few rows and the column names
data.head()
print(data.columns)


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove special characters and digits using regex
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d+', '', text)

    # Tokenize and convert to lowercase
    tokens = text.lower().split()

    # Remove stopwords and apply lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(tokens)

# Check for the correct column name and apply preprocessing
if 'message' in data.columns:
  data['clean_message'] = data['message'].apply(preprocess_text)
elif 'v2' in data.columns:
  # Assuming 'v2' is the correct column name based on the global variables
  data['clean_message'] = data['v2'].apply(preprocess_text)
else:
  print("Message column not found. Please check the CSV file.")

# Check the cleaned data
data.head()
# Use TF-IDF for text vectorization
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(data['clean_message']).toarray()



Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [9]:
# Target variable (spam = 1, not spam = 0)
y = data['v1'].map({'spam': 1, 'ham': 0})  # assuming 'ham' is non-spam and 'v1' is the label column

# Check the shape of the transformed data
X.shape
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the train and test sets
X_train.shape, X_test.shape
# Initialize the Naive Bayes model
model = MultinomialNB()

# Train the model using the training data
model.fit(X_train, y_train)


MultinomialNB()

In [10]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 97.58%
Confusion Matrix:
 [[964   1]
 [ 26 124]]


In [11]:
# Function to predict new emails
def classify_email(text):
    # Preprocess the new text
    processed_text = preprocess_text(text)

    # Vectorize the text using the fitted TF-IDF vectorizer
    vectorized_text = tfidf.transform([processed_text])

    # Predict the label (spam = 1, not spam = 0)
    prediction = model.predict(vectorized_text)

    return 'Spam' if prediction == 1 else 'Not Spam'

# Test with a new email
new_email = "Congratulations! You have won $1000. Click here to claim."
result = classify_email(new_email)
print(f"The email is classified as: {result}")


The email is classified as: Spam
